[`原文档地址`](https://hyper.ai/cn/tutorials/46989)
# 使用 PyTorch 实现深度学习

本教程的目标：

- 理解在 PyTorch 中如何使用张量和构建神经网络。

- 训练一个小型神经网络对图像进行分类

### 目录：
- 张量
- 自动微分
- 构建神经网络

## 一 、张量
张量是一种特殊的数据结构，与数组和矩阵非常相似。在 PyTorch 中，我们使用张量对模型的输入和输出以及模型的参数进行编码。

张量类似于 NumPy 的 ndarray 对象，只是张量可以在 GPU 或其他专用硬件上运行以加速计算。

In [1]:
import torch
import numpy as np

### Z1 张量的初始化

### 1 从数据创建
张量可以直接从数据中创建。PyTorch 将自动推断数据类型。

In [2]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

### 2 从 NumPy 数组创建

In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

### 3 从另一个张量创建
新张量将默认保留旧张量的属性（shape, datatype）。

In [3]:
x_ones = torch.ones_like(x_data) # 保留 x_data 的属性，创建元素值全为 1 的张量
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # 重写 x_data 的 datatype，创建元素值为随机数的张量
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.3243, 0.2383],
        [0.7441, 0.6572]]) 



### 4 使用随机值或常量值创建
shape 是由张量维度组成的元组。

In [5]:
shape = (2,3)
rand_tensor = torch.rand(shape) #张量元素值为随机数
ones_tensor = torch.ones(shape) #张量元素值全为 1
zeros_tensor = torch.zeros(shape) #张量元素值全为 0

### Z2 张量属性
张量属性包括张量的形状(shape)、数据类型(datatype)和存储设备(device)。       
tensor.shape | tensor.dtype | tensor.device

### Z3 张量运算
包括转置、索引、切片、数学运算、线性代数、随机抽样等。详细参看 [`官方文档`](https://docs.pytorch.org/docs/stable/torch.html)

In [9]:
tensor = torch.rand(3,4)
if torch.cuda.is_available():
    tensor = tensor.to('cuda')

### 1 索引和切片

In [11]:
tensor = torch.ones(4, 4)
tensor[:,1] = 0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


### 2 连接
torch.cat 可以沿给定的维度方向连接输入张量。torch.stack 是另一种张量连接方式，它沿一个新维度连接输入张量。

In [14]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1, t1.shape)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]]) torch.Size([4, 12])


### 3 张量乘法

In [15]:
#计算对应元素的乘积
print(f"tensor.mul(tensor) \n {tensor.mul(tensor)} \n")
#替代语法:
print(f"tensor * tensor \n {tensor * tensor}\n")

#两个张量之间的矩阵乘法
print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n")
#替代语法:
print(f"tensor @ tensor.T \n {tensor @ tensor.T}\n")

tensor.mul(tensor) 
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor * tensor 
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

tensor.matmul(tensor.T) 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]]) 

tensor @ tensor.T 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])



### 4 原地操作
带有 _ 后缀的操作将在原地操作，即直接在原本的内存上改变值。例如：x.copy_(y)，x.t_() 将直接更改x。       
原地操作可以节省内存，但在计算梯度时由于会立即丢失历史记录而导致问题，因此，计算梯度时不鼓励使用。

In [16]:
print(tensor, "\n")
tensor.add_(1)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[2., 1., 2., 2.],
        [2., 1., 2., 2.],
        [2., 1., 2., 2.],
        [2., 1., 2., 2.]])


## 二 、自动微分
torch.autograd 是 PyTorch 的自动微分技术，用于帮助神经网络训练。        
神经网络(Neural Networks,NNs)是对输入数据操作的嵌套函数的集合。这些函数由参数（包括权重和偏差）确定，这些参数在 PyTorch 中都存储为张量形式。

### 1 PyTorch中的用法
从 torchvision 加载预训练的 resnet18 模型，同时使用一个随机数据张量来表示一个具有 3 个通道的高度和宽度均为 64 的图像，其相应的标签初始化为随机值。

In [17]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

/home/lllsh/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lllsh/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
prediction = model(data) #前向传播

当我们调用误差张量上的 .backward() 时，反向传播启动，torch.autograd 将自动计算模型里每个参数的梯度并将其存储在参数的 .grad 属性中。

In [22]:
loss = (prediction - labels).sum()
loss.backward()

接下来，我们使用优化器优化参数，在示例中，我们使用 SGD，并设置学习率为 0.01，动量为 0.9。

In [23]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

最后，我们调用 .step() 来启动梯度下降算法，优化器将根据参数 .grad 中存储的梯度来优化每个参数。

In [24]:
optim.step() #梯度下降

## 三、神经网络
更新网络参数，通常使用的更新规则为：参数 = 参数 - 学习率 * 梯度

### 1 定义网络

torch.nn.functional (通常缩写为 F) 是 PyTorch 中的一个模块，提供神经网络的函数式 API。它包含各种操作函数，如激活函数、池化、损失函数等，这些函数不需要创建类实例，直接调用。

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 个输入通道，6 个输出通道，5 x 5 平方卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #仿射运算：y=Wx+b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5 * 5 来自图像尺寸
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #使用（2，2）窗口最大池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #如果大小为正方形，则可以使用单个数字指定
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) #展平除批量维度以外的所有维度
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只需定义前向函数，反向函数（计算梯度）将使用 autograd 自动定义。我们可以在正向函数中使用任何张量运算。

模型的可学习参数由 net.parameters() 返回

In [28]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  #conv1权重

10
torch.Size([6, 1, 5, 5])


### 2 损失函数与优化器

当我们使用神经网络时，我们会需要使用各种不同的更新规则，如 SGD、Nesterov SGD、Adam、RMSProp 等。为了实现这一点，PyTorch 构建了一个小软件包：torch.optim，它实现了所有这些方法。

In [ ]:
import torch.optim as optim

#创建优化器
optimizer = optim.SGD(net.parameters(), lr=0.01)

criterion = nn.MSELoss()
target = torch.randn(10)  #假设的目标

#在训练循环中：
optimizer.zero_grad()   #初始化梯度
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    #更新权重

一句话描述训练过程：只需在数据迭代器上循环将数据正向输入到网络并反向传播，更新权重

In [ ]:
for epoch in range(2):  #在数据集上循环多次

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 1):
        #获取输入；数据是[输入、标签]列表
        inputs, labels = inputs.to(device), labels.to(device)
        #初始化参数梯度
        optimizer.zero_grad()

        #前向传播、反向传播、更新权重
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #打印统计数据
        running_loss += loss.item()
        if i % 2000 == 199:    #每 2000个批量打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

保存训练的模型

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)